### 英文句子中的词性标注
名词，动词等


In [12]:
import numpy as np

# 初始化词典
tag2id, id2tag = {},{}
word2id, id2word = {},{}
# 建立词典
for line in open("../data/traindata.txt"):
    items = line.split("/")
    # items[1] 后面后一个换行符 \n 要去掉
    word, tag = items[0], items[1].rstrip()
    if word not in word2id:
        word2id[word]=len(word2id)
        id2word[len(id2word)] = word
    if tag not in tag2id:
        tag2id[tag] = len(tag2id)
        id2tag[len(id2tag)] = tag


In [13]:
# 初始化参数
M = len(word2id)
N = len(tag2id)

pi = np.zeros(N)    # 初始概率
B = np.zeros((N,M))    # 发射
A = np.zeros((N,N))    # 状态转移

In [14]:
# 统计参数
prev_tag = ""
for line in open("../data/traindata.txt"):
    items = line.split("/")
    # 获取词和标签的ID
    wordId, tagId = word2id[items[0]], tag2id[items[1].rstrip()]
    if prev_tag == "":
        pi[tagId] += 1
        # 计算发射概率的参数，词和标签的对应次数
        B[tagId][wordId] += 1
    else:
        B[tagId][wordId] += 1
        # 转移概率的参数
        A[tag2id[prev_tag]][tagId] += 1
    # 句末重置prev
    if items[0] == ".":
        prev_tag = ""
    else:
        prev_tag = items[1].rstrip()

In [15]:
# 转化成概率的形式，做归一化
pi = pi/sum(pi)
for i in range(N):
    A[i] /= sum(A[i])
    B[i] /= sum(B[i])

In [16]:
# print(f"pi:{pi}")
def log(v):
    if v == 0:
        return np.log(v+0.000001)
    return np.log(v)

In [17]:
# 维特比算法
def viterbi(x,pi,A, B):
    x = [word2id[word] for word in x.split(" ")]
    T = len(x)
    dp = np.zeros((T,N))    # 动态规划组
    ptr = np.zeros((T,N),dtype=int)    # 保存最大路径对应得下标，也就是结果
    for j in range(N):
        # 对数相加等于真数相乘
        # 等于初始概率x发射概率
        dp[0][j] = log(pi[j]) + log(B[j][x[0]])
    for i in range(1,T):
        for j in range(N):
            dp[i][j] = -9999999
            for k in range(N):    # 这里时算上一个列
                score = dp[i-1][k]+log(A[k][j])+log(B[j][x[i]])
                if score > dp[i][j]:
                    dp[i][j] = score
                    ptr[i][j] = k
    best_seq = [0]*T
    print(best_seq)
    best_seq[T-1] = np.argmax(dp[T-1])
    for i in range(T-2,-1,-1):
        best_seq[i] = ptr[i+1][best_seq[i+1]]
    for i in range(len(best_seq)):
        print(id2tag[best_seq[i]])

In [18]:
x = "I like to play ball"
viterbi(x, pi, A, B)

[0, 0, 0, 0, 0]
PRP
VBP
TO
VB
NN
